## <b>Temporal analysis</b>
<i>This script can be used to analyse the created topics and calculated sentiments over time. </i>


In [42]:
#Import needed libraries
import pandas as pd

#Read the data with pandas
outputgermany = pd.read_csv('Data\\sentimenttopic70_tweets_germany.csv')
tweets_germany = pd.DataFrame(outputgermany)

#Convert DateTime column to two columns containing the date and the time
tweets_germany[['Date', 'Time']] = tweets_germany['t_datetime_goed'].str.split(expand=True)

#Add column with month_year to dataframe
tweets_germany['month_year'] = pd.to_datetime(tweets_germany['Date'], format = '%d-%m-%y').dt.to_period('m')

In [65]:
#Create dataframes of the topics
coronapolicies = tweets_germany[tweets_germany['nametopic'] == 'coronapolicies']
lockdownactivities = tweets_germany[tweets_germany['nametopic'] == 'lockdownactivities']
prevention = tweets_germany[tweets_germany['nametopic'] == 'prevention']

Create graph with the frequency of tweets for each topic

In [44]:
import plotly_express as px
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')

#Add topic names to DataFrame
conditions = [
    (tweets_germany['Topic'] == -1),
    (tweets_germany['Topic'] == 0) | (tweets_germany['Topic'] == 2)| (tweets_germany['Topic'] == 4) | (tweets_germany['Topic'] == 6) | (tweets_germany['Topic'] == 13), 
    (tweets_germany['Topic'] == 1) | (tweets_germany['Topic'] == 5)| (tweets_germany['Topic'] == 10)  | (tweets_germany['Topic'] == 14) | (tweets_germany['Topic'] == 15), 
    (tweets_germany['Topic'] == 3) | (tweets_germany['Topic'] == 7)| (tweets_germany['Topic'] == 8) | (tweets_germany['Topic'] == 9) | (tweets_germany['Topic'] == 11)| (tweets_germany['Topic'] == 12)
    ]

values = ['outlier', 'coronapolicies','prevention','lockdownactivities']
tweets_germany['nametopic'] = np.select(conditions, values)

def frequencytweets (topicname, name, title, pngname):
    df = tweets_germany[tweets_germany['nametopic'] == topicname]
    
    name = df['month_year'].value_counts().rename_axis('Month').reset_index(name='Counts').astype(str).sort_values('Month')
    name['Counts'] = name['Counts'].astype(float)

    fig = px.line(name, x="Month", y="Counts", title=title)
    fig.write_image(pngname, width=1000, height=450)

frequencytweets('coronapolicies', 'coronapoliciesvalues', 'Number of tweets per month for topic "Corona and policies"', 'Figures\\frequencycoronapolicies.png')
frequencytweets('lockdownactivities', 'lockdownactivitiesvalues', 'Number of tweets per month for topic "Lockdown activities"', 'Figures\\frequencylockdownactivities.png')
frequencytweets('prevention', 'preventionvalues', 'Number of tweets per month for topic "Prevention"', 'Figures\\frequencyprevention.png')

Get frequency of all topics in one graph

In [45]:
tweets_germany['count'] = 1
alltopics = tweets_germany.groupby(['month_year','nametopic']).count()['count'].unstack().reset_index()
alltopics['month_year'] = alltopics['month_year'].astype(str)
fig = px.line(alltopics, x="month_year", y=['coronapolicies','lockdownactivities', 'prevention'],  labels={"month_year": 'Month', "value":"Frequency", "variable":"Label", 'coronapolicies':"Corona and policies", "prevention":"Prevention","lockdownactivities":"Lockdown activities"},title='Frequency of tweets per topic')
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.96,
    xanchor="right",
    x=0.99
))
legend = {'coronapolicies':"Corona and policies", "prevention":"Prevention","lockdownactivities":"Lockdown activities"}
fig.write_image('Figures\\frequencyalltopics.png', width=1000, height=450)

Total percentage sentiment over time

In [ ]:
import plotly_express as px

totalsentiment = tweets_germany.groupby(['month_year', 'label']).count()['count'].unstack().reset_index().sort_values('month_year')
totalsentiment['month_year'] = totalsentiment['month_year'].astype(str)
totalsentiment['sum'] = totalsentiment['Negative'] + totalsentiment['Neutral'] + totalsentiment['Positive']

totalsentiment['PCT_Negative'] = (totalsentiment.Negative / totalsentiment['sum'].sum()) * 100
totalsentiment['PCT_Neutral'] = (totalsentiment.Neutral / totalsentiment['sum'].sum()) * 100
totalsentiment['PCT_Positive'] = (totalsentiment.Positive / totalsentiment['sum'].sum()) * 100

fig = px.line(totalsentiment, x='month_year', y=["PCT_Negative", "PCT_Neutral", "PCT_Positive"], color_discrete_sequence=['red', 'grey', 'green'], labels={"month_year": 'Month', "value":"Percentage", "variable":"Label", 'PCT_Negative':"Negative", "PCT_Neutral":"Neutral","PCT_Positive":"Positive"}, title="Percentage of tweets per sentiment over time")
fig.update_layout(legend=dict(
        yanchor="top",
        y=0.96,
        xanchor="right",
        x=0.99
))
legend = {'PCT_Negative':"Negative", "PCT_Neutral":"Neutral","PCT_Positive":"Positive"}
fig.for_each_trace((lambda t: t.update(name = legend[t.name])))
fig.update_layout(yaxis_range=[0,13])
fig.show()
fig.write_image("Figures\\totalpctsentimentovertime.png", width=1000, height=450)

Get percentage of tweets per topic in one graph

In [ ]:
alltopics = tweets_germany.groupby(['month_year','nametopic']).count()['count'].unstack().reset_index()
alltopics['month_year'] = alltopics['month_year'].astype(str)
alltopics['sum'] = alltopics['coronapolicies'] + alltopics['lockdownactivities'] + alltopics['prevention'] 

alltopics['PCT_coronapolicies'] = (alltopics.coronapolicies / alltopics['sum'].sum()) * 100
alltopics['PCT_prevention'] = (alltopics.prevention / alltopics['sum'].sum()) * 100
alltopics['PCT_lockdownactivities'] = (alltopics.lockdownactivities / alltopics['sum'].sum()) * 100

fig = px.line(alltopics, x='month_year', y=['PCT_coronapolicies','PCT_lockdownactivities', 'PCT_prevention'], labels={"month_year": 'Month', "value":"Percentage", "variable":"Label", 'PCT_coronapolicies':"Corona and policies", "PCT_prevention":"Prevention","PCT_lockdownactivities":"Lockdown activities"}, title='Percentage of tweets per topic')
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.96,
    xanchor="right",
    x=0.99
))
legend = {'PCT_coronapolicies':"Corona and policies", "PCT_prevention":"Prevention","PCT_lockdownactivities":"Lockdown activities"}
fig.for_each_trace((lambda t: t.update(name = legend[t.name])))
fig.update_layout(yaxis_range=[0,13])
fig.show()
fig.write_image('Figures\PCT_alltopicsfrequency.png', width=1000, height=450)

Get percentage of total tweets per sentiment per topic in graph

In [85]:
events = pd.DataFrame()
events['event'] = ['Start pandemic', 'Less restrictions', 'New years even speech Merkel', 'Further restrictions', 'Closing of retail', 'Prolongation restrictions','Corona speech Merkel', 'Mask obligation', 'Start vaccinations', '3G-rules', 'Nationwide 2G-rules and contact restrictions']
events['Month'] = ['03-2020', '04-2020', '01-2021', '09-2021', '12-2021', '04-2021', '03-2020', '04-2020', '12-2021', '08-2021', '12-2021']
events['Month'] = pd.to_datetime(events['Month'], format = '%m-%Y').dt.to_period('m')

In [176]:
import datetime
def events_coronapolicies (topicname, title, pngname):
    topictweets = tweets_germany[tweets_germany['nametopic'] == topicname]
    
    topictweets['count'] = 1
    topicsentiment = topictweets.groupby(['month_year', 'label']).count()['count'].unstack().reset_index().sort_values('month_year')
    topicsentiment['month_year'] = totalsentiment['month_year'].astype(str)
    
    topicsentiment = topicsentiment.fillna(0)

    topicsentiment['sum'] = topicsentiment['Negative'] + topicsentiment['Neutral'] + topicsentiment['Positive']

    topicsentiment['PCT_Negative'] = (topicsentiment.Negative / topicsentiment['sum'].sum()) * 100
    topicsentiment['PCT_Neutral'] = (topicsentiment.Neutral / topicsentiment['sum'].sum()) * 100
    topicsentiment['PCT_Positive'] = (topicsentiment.Positive / topicsentiment['sum'].sum()) * 100

    fig = px.line(topicsentiment, x='month_year', y=["PCT_Negative", "PCT_Neutral", "PCT_Positive"], color_discrete_sequence=['red', 'grey', 'green'], labels={"month_year": 'Month', "value":"Percentage", "variable":"Label", 'PCT_Negative':"Negative", "PCT_Neutral":"Neutral","PCT_Positive":"Positive"}, title=title)
    fig.add_vline(x=datetime.datetime.strptime("2020-03", "%Y-%m").timestamp() * 1000 ,  line_width=1, line_dash="dash", line_color='black', annotation_text='Start pandemic', annotation_position="top left")
    fig.add_vline(x=datetime.datetime.strptime("2020-04", "%Y-%m").timestamp() * 1000 ,  line_width=1, line_dash="dash", line_color='black', annotation_text='Less restrictions')
    fig.add_vline(x=datetime.datetime.strptime("2020-12", "%Y-%m").timestamp() * 1000 ,  line_width=1, line_dash="dash", line_color='black', annotation_text='New years eve<br>speech Merkel')
    fig.add_vline(x=datetime.datetime.strptime("2021-08", "%Y-%m").timestamp() * 1000 ,  line_width=1, line_dash="dash", line_color='black', annotation_text='3G-rules', annotation_position="top left")
    fig.add_vline(x=datetime.datetime.strptime("2021-11", "%Y-%m").timestamp() * 1000 ,  line_width=1, line_dash="dash", line_color='black', annotation_text='Nationwide<br>2G-rules<br>and<br>contact<br>restrictions', annotation_position="top left")
    legend = {'PCT_Negative':"Negative", "PCT_Neutral":"Neutral","PCT_Positive":"Positive"}
    fig.for_each_trace((lambda t: t.update(name = legend[t.name])))
    fig.update_layout(yaxis_range=[0,13])
    fig.show()
    fig.write_image(pngname, width=1000, height=450)

events_coronapolicies('coronapolicies', "Corona and policies: Percentage of tweets per sentiment over time",'Figures\events_pctsentimentcoronapoliciesovertime.png')

In [182]:
import datetime
def events_lockdown (topicname, title, pngname):
    topictweets = tweets_germany[tweets_germany['nametopic'] == topicname]
    
    topictweets['count'] = 1
    topicsentiment = topictweets.groupby(['month_year', 'label']).count()['count'].unstack().reset_index().sort_values('month_year')
    topicsentiment['month_year'] = totalsentiment['month_year'].astype(str)
    
    topicsentiment = topicsentiment.fillna(0)

    topicsentiment['sum'] = topicsentiment['Negative'] + topicsentiment['Neutral'] + topicsentiment['Positive']

    topicsentiment['PCT_Negative'] = (topicsentiment.Negative / topicsentiment['sum'].sum()) * 100
    topicsentiment['PCT_Neutral'] = (topicsentiment.Neutral / topicsentiment['sum'].sum()) * 100
    topicsentiment['PCT_Positive'] = (topicsentiment.Positive / topicsentiment['sum'].sum()) * 100

    fig = px.line(topicsentiment, x='month_year', y=["PCT_Negative", "PCT_Neutral", "PCT_Positive"], color_discrete_sequence=['red', 'grey', 'green'], labels={"month_year": 'Month', "value":"Percentage", "variable":"Label", 'PCT_Negative':"Negative", "PCT_Neutral":"Neutral","PCT_Positive":"Positive"}, title=title)
    fig.add_vline(x=datetime.datetime.strptime("2020-03", "%Y-%m").timestamp() * 1000 ,  line_width=1, line_dash="dash", line_color='black', annotation_text='Start pandemic', annotation_position="top left")
    fig.add_vline(x=datetime.datetime.strptime("2020-09", "%Y-%m").timestamp() * 1000 ,  line_width=1, line_dash="dash", line_color='black', annotation_text='Further restrictions', annotation_position="top left")
    fig.add_vline(x=datetime.datetime.strptime("2020-12", "%Y-%m").timestamp() * 1000 ,  line_width=1, line_dash="dash", line_color='black', annotation_text='Closing of<br>retail', annotation_position="top left")
    fig.add_vline(x=datetime.datetime.strptime("2021-04", "%Y-%m").timestamp() * 1000 ,  line_width=1, line_dash="dash", line_color='black', annotation_text='Prolongation<br>restrictions', annotation_position="top left")
    fig.add_vline(x=datetime.datetime.strptime("2021-08", "%Y-%m").timestamp() * 1000 ,  line_width=1, line_dash="dash", line_color='black', annotation_text='3G-rules', annotation_position="top left")
    fig.add_vline(x=datetime.datetime.strptime("2021-11", "%Y-%m").timestamp() * 1000 ,  line_width=1, line_dash="dash", line_color='black', annotation_text='Nationwide<br>2G-rules<br>and<br>contact<br>restrictions', annotation_position="top left")
    legend = {'PCT_Negative':"Negative", "PCT_Neutral":"Neutral","PCT_Positive":"Positive"}
    fig.for_each_trace((lambda t: t.update(name = legend[t.name])))
    fig.update_layout(yaxis_range=[0,13])
    fig.show()
    fig.write_image(pngname, width=1000, height=450)

events_lockdown('lockdownactivities', "Lockdown activities: Percentage of tweets per sentiment over time",'Figures\events_pctsentimentlockdownactivitiesovertime.png')

In [ ]:
import datetime
def events_prevention (topicname, title, pngname):
    topictweets = tweets_germany[tweets_germany['nametopic'] == topicname]
    
    topictweets['count'] = 1
    topicsentiment = topictweets.groupby(['month_year', 'label']).count()['count'].unstack().reset_index().sort_values('month_year')
    topicsentiment['month_year'] = totalsentiment['month_year'].astype(str)
    
    topicsentiment = topicsentiment.fillna(0)

    topicsentiment['sum'] = topicsentiment['Negative'] + topicsentiment['Neutral'] + topicsentiment['Positive']

    topicsentiment['PCT_Negative'] = (topicsentiment.Negative / topicsentiment['sum'].sum()) * 100
    topicsentiment['PCT_Neutral'] = (topicsentiment.Neutral / topicsentiment['sum'].sum()) * 100
    topicsentiment['PCT_Positive'] = (topicsentiment.Positive / topicsentiment['sum'].sum()) * 100

    fig = px.line(topicsentiment, x='month_year', y=["PCT_Negative", "PCT_Neutral", "PCT_Positive"], color_discrete_sequence=['red', 'grey', 'green'], labels={"month_year": 'Month', "value":"Percentage", "variable":"Label", 'PCT_Negative':"Negative", "PCT_Neutral":"Neutral","PCT_Positive":"Positive"}, title=title)
    fig.add_vline(x=datetime.datetime.strptime("2020-03", "%Y-%m").timestamp() * 1000 ,  line_width=1, line_dash="dash", line_color='black', annotation_text='Corona speech Merkel', annotation_position="top left")
    fig.add_vline(x=datetime.datetime.strptime("2020-09", "%Y-%m").timestamp() * 1000 ,  line_width=1, line_dash="dash", line_color='black', annotation_text='Mask obligation', annotation_position="top left")
    fig.add_vline(x=datetime.datetime.strptime("2020-12", "%Y-%m").timestamp() * 1000 ,  line_width=1, line_dash="dash", line_color='black', annotation_text='Start vaccinations')
    fig.add_vline(x=datetime.datetime.strptime("2021-08", "%Y-%m").timestamp() * 1000 ,  line_width=1, line_dash="dash", line_color='black', annotation_text='3G-rules', annotation_position="top left")
    fig.add_vline(x=datetime.datetime.strptime("2021-11", "%Y-%m").timestamp() * 1000 ,  line_width=1, line_dash="dash", line_color='black', annotation_text='Nationwide<br>2G-rules<br>and<br>contact<br>restrictions', annotation_position='top left')
    legend = {'PCT_Negative':"Negative", "PCT_Neutral":"Neutral","PCT_Positive":"Positive"}
    fig.for_each_trace((lambda t: t.update(name = legend[t.name])))
    fig.update_layout(yaxis_range=[0,13])
    fig.show()
    fig.write_image(pngname, width=1000, height=450)

events_prevention('prevention', "Prevention: Percentage of tweets per sentiment over time",'Figures\events_pctsentimentcpreventionovertime.png')

In [ ]:
def pct_sentimentpertopic (topicname, title, pngname):
    topictweets = tweets_germany[tweets_germany['nametopic'] == topicname]
    
    topictweets['count'] = 1
    topicsentiment = topictweets.groupby(['month_year', 'label']).count()['count'].unstack().reset_index().sort_values('month_year')
    topicsentiment['month_year'] = totalsentiment['month_year'].astype(str)
    
    topicsentiment = topicsentiment.fillna(0)

    topicsentiment['sum'] = topicsentiment['Negative'] + topicsentiment['Neutral'] + topicsentiment['Positive']

    topicsentiment['PCT_Negative'] = (topicsentiment.Negative / topicsentiment['sum'].sum()) * 100
    topicsentiment['PCT_Neutral'] = (topicsentiment.Neutral / topicsentiment['sum'].sum()) * 100
    topicsentiment['PCT_Positive'] = (topicsentiment.Positive / topicsentiment['sum'].sum()) * 100

    fig = px.line(topicsentiment, x='month_year', y=["PCT_Negative", "PCT_Neutral", "PCT_Positive"], color_discrete_sequence=['red', 'grey', 'green'], labels={"month_year": 'Month', "value":"Percentage", "variable":"Label", 'PCT_Negative':"Negative", "PCT_Neutral":"Neutral","PCT_Positive":"Positive"}, title=title)
    fig.update_layout(legend=dict(
        yanchor="top",
        y=0.96,
        xanchor="right",
        x=0.99
    ))
    legend = {'PCT_Negative':"Negative", "PCT_Neutral":"Neutral","PCT_Positive":"Positive"}
    fig.for_each_trace((lambda t: t.update(name = legend[t.name])))
    fig.update_layout(yaxis_range=[0,13])
    fig.show()
    fig.write_image(pngname, width=1000, height=450)

In [ ]:
pct_sentimentpertopic('coronapolicies', "Corona and policies: Percentage of tweets per sentiment over time",'Figures\pctsentimentcoronapoliciesovertime.png')
pct_sentimentpertopic('lockdownactivities', "Lockdown activities: Percentage of tweets per sentiment over time",'Figures\pctsentimentlockdownactivitiesovertime.png')
pct_sentimentpertopic('prevention', "Prevention: Percentage of tweets per sentiment over time",'Figures\pctsentimentcpreventionovertime.png')

Get percentage of tweets of each sentiment per topic over time

In [ ]:
#Create dataframe with the month, topics, label and counts per label
sentimentalltopics = tweets_germany.groupby(['month_year','nametopic','label']).count()['count'].unstack().reset_index()
sentimentalltopics['month_year'] = sentimentalltopics['month_year'].astype(str)
sentimentalltopics  = ssentimentalltopics['month_year'] = sentimentalltopics['month_year'].astype(str)entimentalltopics.fillna(0)

#Create sum of tweets per month per topic
sentimentalltopics['sum'] = sentimentalltopics['Negative'] + sentimentalltopics['Positive'] + sentimentalltopics['Neutral'] 

#Calculate percentage of sentiment per month
sentimentalltopics['PCT_Negative'] = (sentimentalltopics.Negative / sentimentalltopics['sum']) * 100
sentimentalltopics['PCT_Neutral'] = (sentimentalltopics.Neutral / sentimentalltopics['sum']) * 100
sentimentalltopics['PCT_Positive'] = (sentimentalltopics.Positive / sentimentalltopics['sum']) * 100

#Calculate percentage of negative tweets per topic (from the total tweets in that topic)
#First, add total tweets per topic in new column
sentimentalltopics.loc[sentimentalltopics['nametopic'] == 'coronapolicies', 'coronasum'] = '3310'
sentimentalltopics.loc[sentimentalltopics['nametopic'] == 'lockdownactivities', 'coronasum'] = '2665'
sentimentalltopics.loc[sentimentalltopics['nametopic'] == 'prevention', 'coronasum'] = '2480'
sentimentalltopics['coronasum'] = sentimentalltopics['coronasum'].astype('float')

sentimentalltopics['topicNegative'] = (sentimentalltopics.Negative / sentimentalltopics.coronasum) * 100
sentimentalltopics['topicNeutral'] = (sentimentalltopics.Neutral / sentimentalltopics.coronasum) * 100
sentimentalltopics['topicPositive'] = (sentimentalltopics.Positive / sentimentalltopics.coronasum) * 100

#Dataframe without the outlier category
sentimenttopics = sentimentalltopics.loc[(sentimentalltopics['nametopic'] == 'lockdownactivities') | (sentimentalltopics['nametopic'] == 'coronapolicies') | (sentimentalltopics['nametopic'] == 'prevention')]
sentimenttopics['month_year'] = sentimenttopics['month_year'].astype('datetime64')
#Change names of topics
sentimenttopics["nametopic"] = sentimenttopics["nametopic"].map({'coronapolicies': "Corona and policies", 'lockdownactivities': "Lockdown activities", 'prevention': "Prevention"})


In [ ]:
#Create graph with the percentage of sentiment per topic in each month
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

grid = sns.FacetGrid(sentimenttopics, col='nametopic', hue='nametopic', col_wrap=2, height=5, legend_out=True )

grid = grid.map(plt.plot, 'month_year', 'topicNegative', color='red')
grid = grid.map(plt.plot, 'month_year', 'topicPositive', color='green')
grid = grid.map(plt.plot, 'month_year', 'topicNeutral', color='blue')

grid = grid.set_titles(col_template="{col_name}", fontsize=20,fontweight='bold')

grid.fig.suptitle('Percentage of tweets of each sentiment per topic over time', fontsize=20)

for ax in grid.axes.ravel():
    ax.legend(labels=['Negative', 'Positive', 'Neutral'], fontsize=13)
    ax.margins(0.00001)
    ax.tick_params(labelbottom=True, labelleft=True)

grid.set_xlabels('Date', fontsize=14)
grid.set_ylabels('Percentage', fontsize=14)

plt.subplots_adjust(top=0.92)
grid = grid.figure.set_size_inches(20,15)

In [ ]:
#Create figure containing the percentage of negative and positive tweets of all tweets in a month, per topic

grid = sns.FacetGrid(sentimenttopics, col='nametopic', hue='nametopic', col_wrap=2, height=5, legend_out=True )

grid = grid.map(plt.plot, 'month_year', 'PCT_Negative', color='r')
grid = grid.map(plt.plot, 'month_year', 'PCT_Positive', color='k', linestyle = 'dashed' )


grid = grid.map(plt.fill_between, 'month_year', 'PCT_Negative', alpha=0.2, color='r').set_titles("{col_name} nametopic")
grid = grid.map(plt.fill_between, 'month_year', 'PCT_Positive', alpha=0.2, hatch = '//', color='none').set_titles("{col_name} nametopic")

grid = grid.set_titles(col_template="{col_name}", fontsize=20,fontweight='bold')


grid.fig.suptitle('Percentage of negative and positive tweets per topic (of all tweets in a month)', fontsize=20,)

for ax in grid.axes.ravel():
    ax.legend(labels=['Negative', 'Positive'], fontsize=13)
    ax.margins(0.00001)
    ax.tick_params(labelbottom=True)

grid.set_xlabels('Date', fontsize=14)
grid.set_ylabels('Percentage', fontsize=14)

plt.subplots_adjust(top=0.92)
grid = grid.figure.set_size_inches(20,15)

Augmented Dickey-Fuller test 

In [59]:
tweets_germany['count'] = 1
tweets_germany['month_year'] = pd.to_datetime(tweets_germany['Date'], format = '%d-%m-%y').dt.to_period('m')

#Create dataframe containing topics
topics = tweets_germany.groupby(['month_year','nametopic']).count()['count'].unstack().reset_index()
#topics = topics.fillna(0)

#Create dataframe containing the sentiments
sentiments = tweets_germany.groupby(['month_year', 'label']).count()['count'].unstack().reset_index()

In [60]:
from statsmodels.tsa.stattools import adfuller

#Label is sentiment or topic
def ADF (data, label):
    series = data.loc[:, label].values
    result = adfuller(series)
    print(label)
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
	    print('\t%s: %.3f' % (key, value))

In [ ]:
ADF(sentiments, 'Positive')
ADF(sentiments, 'Negative')
ADF(sentiments, 'Neutral')

ADF(topics, 'coronapolicies')
ADF(topics, 'lockdownactivities')
ADF(topics, 'prevention')